In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import matplotlib.pyplot as plt
# import seaborn as sns

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.math import confusion_matrix

2024-05-10 00:56:10.054002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 00:56:10.337873: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 00:56:10.370326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-10 00:56:10.370361: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [4]:
train_ds, val_ds = image_dataset_from_directory(
    directory='../raw_data/train',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='both',
    # verbose=True
)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Using 5741 files for validation.


2024-05-10 01:10:00.222216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-10 01:10:00.223144: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-10 01:10:00.223385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hp-ali): /proc/driver/nvidia/version does not exist
2024-05-10 01:10:00.228416: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
test_ds = image_dataset_from_directory(
    directory='../raw_data/test',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=False,
    seed=42,
    # verbose=True
)

Found 7178 files belonging to 7 classes.


In [6]:
pip freeze | grep tensorflow

tensorflow==2.10.0
tensorflow-estimator==2.10.0
tensorflow-io-gcs-filesystem==0.37.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
input_shape=train_ds.element_spec[0].shape[1:]
input_shape

TensorShape([299, 299, 3])

In [8]:
class_names = train_ds.class_names
print(class_names)


['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [9]:
train_ds.element_spec

(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))

In [10]:
for image_batch, labels_batch in val_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


(32, 299, 299, 3)
(32, 7)


In [11]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")


In [12]:
def preprocess_data(img, label):
    return preprocess_input(img), label

preproc_test_ds = test_ds.map(preprocess_data)
preproc_train_ds = train_ds.map(preprocess_data)
preproc_val_ds = val_ds.map(preprocess_data)

In [13]:
# base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)
# base_model.trainable = False

# aug1 = layers.RandomFlip("horizontal")
# aug2 = layers.RandomRotation(0.1)
# # aug3 = layers.RandomZoom(0.1)

# flatten_layer = Flatten()

# dense_layer1 = Dense(700, activation='relu')
# dropout1 = Dropout(0.25)
# # dense_layer2 = Dense(250, activation='relu')
# # dropout2 = Dropout(0.1)
# dense_layer3 = Dense(70, activation='relu')
# # dropout3 = Dropout(0.1)

# prediction_layer = Dense(7, activation='softmax')


# model = Sequential([
#     base_model,
#     aug1,
#     aug2,
# #     aug3,
#     flatten_layer,
#     dense_layer1,
#     dropout1,
#     dense_layer2,
#     dropout2,
#     dense_layer3,
# #     dropout3,
#     prediction_layer
# ])

In [14]:
img_shape = train_ds.element_spec[0].shape[1:]

In [15]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)
base_model.trainable = False

inputs = Input(shape=(img_shape))
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomRotation(0.1)(x)

x = base_model(x)
x = Flatten()(x)

x = Dense(700, activation='relu')(x)
x = Dropout(0.35)(x)
x = BatchNormalization()(x)
x = Dense(350, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
# x = Dense(150, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(75, activation='relu')(x)
x = BatchNormalization()(x)

outputs = Dense(7, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

87910968/87910968 [==============================] - 46s 1us/step


In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 random_flip (RandomFlip)    (None, 299, 299, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 299, 299, 3)      0         
 tion)                                                           
                                                                 
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 700)               91751100  
                                                             

In [17]:
from datetime import datetime

# Format the date and time
formatted_time = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

In [18]:
lr = ReduceLROnPlateau(monitor="val_loss",
                       factor=0.1,
                       patience=2,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}_Inception_V3.keras".format(formatted_time),
                      monitor='val_loss',
                      mode='auto',
                      verbose=0,
                      save_best_only=True)

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 3, 
                   verbose = 1, 
                   restore_best_weights = True)

In [19]:
initial_learning_rate = 0.0001
# lr_schedule = ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=10,
#     decay_rate=0.99,
#     staircase=True)

optimizer = Adam(learning_rate=initial_learning_rate)

In [20]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
from sklearn.utils import class_weight

classes = np.array([label for _, label in train_ds.unbatch()])
integer_classes = np.argmax(classes, axis=1)
weights = class_weight.compute_class_weight('balanced', classes=np.unique(integer_classes), y=integer_classes)
weights_dict = {i: weight for i, weight in enumerate(weights)}

In [ ]:


history = model.fit(preproc_train_ds, 
                    validation_data=preproc_val_ds, 
                    epochs=100,
                    class_weight=weights_dict,
                    callbacks=[es, mcp, lr])

Epoch 1/100
495/718 [===================>..........] - ETA: 48:43 - loss: 2.0346 - accuracy: 0.2670

In [24]:
model.evaluate(preproc_test_ds)

225/225 ━━━━━━━━━━━━━━━━━━━━ 36s 161ms/step - accuracy: 0.4818 - loss: 1.3598


[1.2761727571487427, 0.5256338715553284]

In [26]:
balanced_test_ds = image_dataset_from_directory(
    directory='/kaggle/input/balanced-test/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=True,
    seed=42,
    verbose=True
)

preproc_balanced_test_ds = balanced_test_ds.map(preprocess_data)
model.evaluate(preproc_balanced_test_ds)

Found 700 files belonging to 7 classes.
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 379ms/step - accuracy: 0.5625 - loss: 1.2150


[1.2297910451889038, 0.5471428632736206]

In [ ]:
model.save('Inception_v3.keras')

In [ ]:
!pip freeze | grep tensorflow

In [ ]:
!ls

In [ ]:
loaded_model = load_model('/kaggle/input/inception_v3_2/tensorflow2/inception_v3/1/Inception_v3.keras')
loaded_model.summary()

In [ ]:
loaded_model.evaluate(preproc_test_ds)

In [ ]:
balanced_test_ds = image_dataset_from_directory(
    directory='/kaggle/input/balanced-test/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(299, 299),
    shuffle=False,
    seed=42,
    verbose=True
)

In [ ]:
preproc_balanced_test_ds = balanced_test_ds.map(preprocess_data)
loaded_model.evaluate(preproc_balanced_test_ds)

In [ ]:
predictions = loaded_model.predict(preproc_balanced_test_ds)
predicted_classes = np.argmax(predictions, axis=1)

# Retrieve true labels
true_labels = np.concatenate([y for x, y in preproc_balanced_test_ds], axis=0)

# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels.argmax(axis=1), predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
labels.shape

In [ ]:
labels = []
for images, label_batch in train_ds:
    labels.extend(label_batch.numpy())  # Ensure labels are numpy arrays, not tensors

labels = np.array(labels)  # Convert list of labels to a numpy array

# Compute class weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels.flatten()
)

In [ ]:
for image_batch, labels_batch in val_ds:
  integer_labels = np.argmax(labels_batch, axis=1)
  print(integer_labels)
  break

In [ ]:
# from google.cloud import storage

# def upload_file_to_gcs(bucket_name, source_file_name, destination_blob_name):
#     """Uploads a file to the Google Cloud Storage bucket."""
#     # Set up the client with Google Cloud credentials
#     storage_client = storage.Client.from_service_account_json('path/to/your/service-account-key.json')
    
#     # Get the bucket
#     bucket = storage_client.bucket(bucket_name)
    
#     # Create a blob object from the filepath
#     blob = bucket.blob(destination_blob_name)
    
#     # Upload the file to a destination
#     blob.upload_from_filename(source_file_name)

#     print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# # Example usage
# upload_file_to_gcs('hayd1621', 'local/path/to/your/file', 'dest